In [1]:
import json

In [2]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, ValidationError, validator
from datetime import date,datetime

In [3]:
def complex_date_format(dict_,format_="%m/%d/%Y %H:%M:%S"):
    def converter(obj):
        if isinstance(obj, (datetime, date)):
            return datetime.fromisoformat(obj.isoformat()).strftime(format_)
        raise TypeError (f"{type(obj)} not datetime")
    return json.loads(json.dumps(dict_, default=converter))

In [4]:
class User(BaseModel):
    id: int
    name : str
    signup_ts: Optional[datetime] = None
    friends: List[int] = []
        
    @validator('name')
    def name_must_have_space(cls, v):
        if ' ' not in v:
            raise ValueError('must contain contain a space')
        return v

In [5]:
external_data = {
    'id': '123',
    'name':'hello ',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3']}

In [6]:
user = [User(**external_data).dict() for _ in range(3)]

In [7]:
user[0]

{'id': 123,
 'name': 'hello ',
 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
 'friends': [1, 2, 3]}

In [8]:
complex_date_format(user,"%d-%m-%Y")[0]

{'id': 123, 'name': 'hello ', 'signup_ts': '01-06-2019', 'friends': [1, 2, 3]}

In [9]:
complex_date_format(user)[0]

{'id': 123,
 'name': 'hello ',
 'signup_ts': '06/01/2019 12:22:00',
 'friends': [1, 2, 3]}

In [10]:
data = {
#     'id': '123',
    'name':'hello',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3']
}

In [13]:
try:
    User(**data)
except ValidationError as e:
    err =[]
    for i in json.loads(e.json()):
        err.append({"key":i['loc'][0],"msg":i['msg']})
    print(json.dumps(err,indent=2))


[
  {
    "key": "id",
    "msg": "field required"
  },
  {
    "key": "name",
    "msg": "must contain contain a space"
  }
]
